In [ ]:
# Importing Libraries:

import pandas as pd
import numpy as np

from scipy.stats import shapiro, normaltest, kurtosis, skew
from statsmodels.stats.diagnostic import lilliefors
from statsmodels.api import qqplot

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.base import clone
from sklearn.feature_selection import SelectKBest, SelectFromModel

In [ ]:
# Importing data:

df_raw = pd.read_csv("Solar_Power_Plant_Data.csv")

In [ ]:
# Copy of the dataset:
df = df_raw.copy()

In [ ]:
# Looking at the first 5 rolls:

df.head()

In [ ]:
# Information about the dataset:

df.info()

In [ ]:
# Missing data:

df.isna().sum()

In [ ]:
# Changing Date-Hour(NMT) column type:

df['Date-Hour(NMT)'] = pd.to_datetime(df['Date-Hour(NMT)'], format="%d.%m.%Y-%H:%M")

df.info()

In [ ]:
# Range of datetime column:

print(f"Range of Datetime column: ({df['Date-Hour(NMT)'].min()}) to ({df['Date-Hour(NMT)'].max()})")

In [ ]:
# Setting the Date-Hour(NUMT) as the index:

df.set_index("Date-Hour(NMT)", inplace=True)

In [ ]:
# Descriptive Statistics analysis:

df.describe()

In [ ]:
# Let's change the type of the numeric variables:

df['Sunshine'] = df['Sunshine'].astype("int16")
df['RelativeAirHumidity'] = df['RelativeAirHumidity'].astype("int16")
df['WindSpeed'] = df['WindSpeed'].astype("float32")
df['Radiation'] = df['Radiation'].astype("float32")
df['AirTemperature'] = df['AirTemperature'].astype("float32")

The memory usage of the dataset reduced from 547.6 to 342.2 KB.

In [ ]:
# Information about the dataset:

df.info()

In [ ]:
# Let's see how many data points have negative Radiation:

df[df["Radiation"] < 0].shape

In [ ]:
# Analysing average monthly Radiation and Sunshine:

df["Months"] = df.index.month_name()
df["Month_number"] = df.index.month
monthly_rad_sun = df.groupby(["Month_number", "Months"]).agg({"Radiation":"mean", "Sunshine":"mean"}).\
    droplevel(level="Month_number")


In [ ]:
# Line plot of the Average monthly Radiation and Sushine

fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(13, 5))
fig.suptitle("Average monthly Radiation and Sunshine")
plt.xticks(rotation=45)

sns.lineplot(monthly_rad_sun["Radiation"], ax=ax[0])

sns.lineplot(monthly_rad_sun["Sunshine"], ax=ax[1], c="r");

In [ ]:
# Dropping Months and Month_number columns:

df.drop(columns=["Months", "Month_number"], inplace=True)

Conclusions

- One year of one-hour mesurements.
- As we can see from the statistic summary, there are some negative data points in Radiation column.
- Average Monthly Radiation is positive over the year.
- The increase of the Radiation and Sunshine between April and August may suggest that this location is situated in the Northern Hemisphere.
- Most data in SystemProduction and Sunshine columns may be zero.


### 1) Exploratory Data Analysis

#### 1.1) Distribution

We will look at the histogram of all variables

In [ ]:
# Histograms:

fig, axes = plt.subplots(nrows=3, ncols=3, sharey=False, figsize=(20, 15))
fig.suptitle("Histograms of the Variables")

sns.histplot(df["SystemProduction"].values, ax=axes[0, 0], kde=True)
axes[0, 0].set_title("SystemProduction")

sns.histplot(df["WindSpeed"].values, ax=axes[0, 1], kde=True)
axes[0, 1].set_title("WindSpeed")

sns.histplot(df["Sunshine"].values, ax=axes[0][2], kde=True)
axes[0][2].set_title("Sunshine")

sns.histplot(df["RelativeAirHumidity"].values, ax=axes[1, 0], kde=True)
axes[1, 0].set_title("RelativeAirHumidity")

sns.histplot(df["Radiation"].values, ax=axes[1, 1], kde=True, legend=False)
axes[1, 1].set_title("Radiation")

sns.histplot(df["AirTemperature"].values, ax=axes[1, 2], kde=True)
axes[1, 2].set_title("AirTemperature")

sns.histplot(df["AirPressure"].values, ax=axes[2, 0], kde=True)
axes[2, 0].set_title("AirPressure");

axes[2, 1].set_visible(False)
axes[2, 2].set_visible(False)

Looking at the Histograms, we may think that AirPressure column was drew from a normal distribution. Let's look at the boxplots.

In [ ]:
# BoxPlots:

fig, ax = plt.subplots(3, 3, figsize=(20, 15), sharey=False)
fig.suptitle("Boxplots")

sns.boxplot(y=df["SystemProduction"].values, ax=ax[0, 0])
ax[0, 0].set_xlabel("SystemProduction")

sns.boxplot(y=df["WindSpeed"].values, ax=ax[0, 1])
ax[0, 1].set_xlabel("WindSpeed")

sns.boxplot(y=df["Sunshine"].values, ax=ax[0, 2])
ax[0, 2].set_xlabel("Sunshine")

sns.boxplot(y=df["RelativeAirHumidity"].values, ax=ax[1, 0])
ax[1, 0].set_xlabel("RelativeAirHumidity")

sns.boxplot(y=df["Radiation"].values, ax=ax[1, 1])
ax[1, 1].set_xlabel("Radiation")

sns.boxplot(y=df["AirTemperature"].values, ax=ax[1, 2])
ax[1, 2].set_xlabel("AirTemperature")

sns.boxplot(y=df["AirPressure"].values, ax=ax[2, 0])
ax[2, 0].set_xlabel("AirPressure");

ax[2, 1].set_visible(False)
ax[2, 2].set_visible(False)

#### 1.2) Normality tests

In [ ]:
# QQ Plots:

# Defining subplots:
fig, axe = plt.subplots(3, 3, sharey=False, figsize=(20, 20))
fig.suptitle("Quantile-Quantile Plots")

# Plotting SystemProduction data:
qqplot(df["SystemProduction"], ax=axe[0, 0], line="s");
axe[0, 0].set_title("SystemProduction")

# Plotting WindSpeed data:
qqplot(df["WindSpeed"], ax=axe[0, 1], line="s")
axe[0, 1].set_title("WindSpeed")

# Plotting Sunshine data:
qqplot(df["Sunshine"], ax=axe[0, 2], line="s")
axe[0, 2].set_title("Sunshine")

# Ploting RelativeAirHumidity:
qqplot(df["RelativeAirHumidity"], ax=axe[1, 0], line="s")
axe[1, 0].set_title("RelativeAirHumidity")

# Radiation:
qqplot(df["Radiation"], ax=axe[1, 1], line="s")
axe[1, 1].set_title("Radiation")

# AirTemperature:
qqplot(df["AirTemperature"], ax=axe[1, 2], line="s")
axe[1, 2].set_title("AirTemperature")

# AirPressure
qqplot(df["AirPressure"], ax=axe[2, 0], line="s")
axe[2, 0].set_title("AirPressure");

axe[2, 1].set_visible(False)
axe[2, 2].set_visible(False)

As we can see from the qqplot and the Histogram above, it seems tha AirPressure follows a normal distribution. Let's quantify this assumption using statistical tests for normality.

- H0: Data was drew from a normal distribution.

- H1: Data was not drew from a normal distribution.

OBS: Using level of significance of 5% (alpha).

OBS2: Shapiro wilk p value is an approximate value due to the size of the sample being more than 5000.

In [ ]:
# Function that Calculates Shapiro-Wilk, Lilliefors and D'Agostino_K2 tests:

def normality_tests(df: any):
    tests_names = ["Shapiro-Wilk", "Lilliefors", "D'Agostino_K2"]
    extern_index = np.array(sorted(tests_names*2))
    intern_index = np.array(["statistic", "p-value"]*len(tests_names))

    mult_index = [
        extern_index,
        intern_index
    ]

    results = pd.DataFrame(index=mult_index, columns=df.columns)
   
    for c in df.columns:
        
        # First D'Agostino's K-squared test:
        k2, k2_p = normaltest(df[c])

        # Secondly we will use the Lilliefors test:
        lilliefors_result = lilliefors(df[c])
        ksstat, lilliefours_p = lilliefors_result

        #  We will check the shapiro-Wilk test:
        shapiro_result = shapiro(df[c])
        shapiro_statistic, shapiro_p = shapiro_result.statistic, shapiro_result.pvalue

        results[c] = [k2, k2_p, ksstat, lilliefours_p, shapiro_statistic, shapiro_p]

    return results


# Function that calculates kurtosis and skewness of a dataset:

def kurtosis_skewness(dataset: any):
    index = ["Kurtosis", "Skewness"]
    results = pd.DataFrame(index=index, columns=dataset.columns)
    for c in dataset.columns:
        kurt = kurtosis(dataset[c])
        skewness = skew(dataset[c])
        results[c] = [kurt, skewness]

    return results

In [ ]:
# Normality test results:

normality_tests(df)

In [ ]:
# Kurtosis and Skewness:

kurtosis_skewness(df)

Distribution can be considered normal:

- Kurtosis between (-1, +1).

- Skewness between (-1, +1).

- Histogram and QQ Plot.


#### 1.3) Correlation

In [ ]:
# Correlation Matrix:

def heatmap_cor(df):
    cor = df.corr()
    mascara = np.zeros_like(cor)
    mascara[np.triu_indices_from(mascara)] = True
    sns.heatmap(cor, mask=mascara, cbar=True, annot=True, cmap="crest")
    

heatmap_cor(df)

Conclusions

- Most of the data in SystemProduction is 0 as we expected since median is zero and the variable can't be negative.

- Outliers exist in SystemProduction, AirPressure, RelativeHumidity, Radiation, WindSpeed and Sunshine.

- All statistical tests have shown that the AirPressure not follows a normal distribution. However, all these tests are very sensitive when the sample size is large. So we can't rely on them.

- Histogram, QQ Plot, Kurtosis and Skewness tell us that AirPressue follows a normal distribution. Therefore, we will assume that AirPressure is normally distributed.

- Radiation has the highest positive correlation coefficient associated with the SystemProdution (0.79). It is also hightly correlated with Sushine column.

- RelativeAirHumidity has the lowest correlation coefficient associated with the Target. (-0.55)


### 2) Preprocessing

In [ ]:
# Separating variables (features and target)

X = df.drop(columns="SystemProduction")
y = df["SystemProduction"]

In [ ]:
# Divinding dataset in train and test:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### 2.2) Transformation

In [ ]:
# Function used to evaluate the best algorithms:

def melhor_modelo(X_train, y_train):

    seed = 42
    cv = 5
    score = ['neg_root_mean_squared_error', 'r2']
    result_rmse = {}
    result_r2 = {}

    dicionario = { 
                "Lasso":Lasso(random_state=seed),
                "Ridge":Ridge(random_state=seed),
                "SVR":SVR(),
                "RandomForestR":RandomForestRegressor(random_state=seed),
                "ExtraTreeR":ExtraTreesRegressor(random_state=seed),
                "XGB":XGBRegressor(random_state=42),
                "MLP":MLPRegressor(random_state=42, max_iter=2000)
                 }


    for name, model in dicionario.items():
        k_fold = KFold(n_splits=cv, random_state=seed, shuffle=True)
        result = cross_validate(model, X_train, y_train, cv=k_fold, scoring=score)

        result_rmse[name] = -result['test_neg_root_mean_squared_error']
        result_r2[name] = result['test_r2']
        
        
    result_pd_rmse = pd.DataFrame(data=result_rmse)
    result_pd_r2 = pd.DataFrame(data=result_r2)
    
    return result_pd_rmse, result_pd_r2
    

##### 2.2.1) MinMax Scaler

In [ ]:
# MinMax Scaler Transformation:

min_max = MinMaxScaler()
X_train_min_max = min_max.fit_transform(X_train)
X_test_min_max = min_max.transform(X_test)

In [ ]:
# Best model:

resultado_rms, resultado_r2 = melhor_modelo(X_train_min_max, y_train)

In [ ]:
# PLotting model's performance:

plt.figure(figsize=(15, 8))
plt.title("Models's Performance - MinMax Scaler")
sns.boxplot(resultado_rms);

In [ ]:
# Root mean squared error results:

resultado_rms.describe()

In [ ]:
# R2 results:

resultado_r2.describe()

##### 2.2.1) Standard Scaler

In [ ]:
# Standard Scaler Transformation:

std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

In [ ]:
# Best model:

resultado_rms, resultado_r2 = melhor_modelo(X_train_std, y_train)

In [ ]:
# PLotting model's performance:

plt.figure(figsize=(15, 8))
plt.title("Models's Performance - Standard Scaler")
sns.boxplot(resultado_rms);

In [ ]:
# Root mean squared error results:

resultado_rms.describe()

In [ ]:
# R2 results:

resultado_r2.describe()

##### 2.2.1) Robust Scaler

In [ ]:
# Robust Scaler Transformation:

rob = RobustScaler()
X_train_rb = rob.fit_transform(X_train)
X_test_rb = rob.fit_transform(X_test)

In [ ]:
# Best model:

resultado_rms, resultado_r2 = melhor_modelo(X_train_rb, y_train)

In [ ]:
# PLotting model's performance:

plt.figure(figsize=(15, 8))
plt.title("Models's Performance - Robust Scaler")
sns.boxplot(resultado_rms);

In [ ]:
# Root mean squared error results:

resultado_rms.describe()

In [ ]:
# R2 results:

resultado_r2.describe()

Conclusions

- ExtraTres was the best Algorithm for all of the transformation algorithms.
- The Best Trasformation algorithm was Robust Scaler.

RMSE:
- Average:  775.018174
- Standard Deviation: 37.423964

R2
- Average: 0.734621
- Standard Deviation: 0.012831

### 3) Feature Selection

In [ ]:
# Class that put together many feature selection techniques:

class feature_selector:
    seed = 42
    def __init__(self, X, y) -> None:
        self.X_train = X
        self.y_train = y
    
    def  randomforestR_imp(self) -> None:
        model = RandomForestRegressor(random_state=feature_selector.seed)
        model.fit(self.X_train, self.y_train)
        series = pd.Series(index=model.feature_names_in_, data=model.feature_importances_).sort_values(ascending=False)

        # Plotting RandomForest Regression Importance:
        plt.title("RandomForest Importance")
        sns.barplot(x=series.values, y=series.index)
        

    def xgbR_imp(self) -> None:
        model = XGBRegressor(random_state=feature_selector.seed)
        model.fit(self.X_train, self.y_train)
        series = pd.Series(index=model.feature_names_in_, data=model.feature_importances_).sort_values(ascending=False)

        # Plotting XGboost Regression Feature Importance:
        plt.title("XGBoost Feature Importance")
        sns.barplot(y=series.index, x=series.values)


    # Construção de Seletor de variáveis univariavel
    def univariate(self, statistic, n, X_test=None) -> None:
        selector = SelectKBest(score_func=statistic, k=n)
        X_selected = selector.fit_transform(self.X_train)
        
        if X_test != None:
            X_test_new = selector.transform(X_test)
            return X_selected, X_test_new

        pass
        

In [ ]:
feature = feature_selector(X_train, y_train)

In [ ]:
feature.randomforestR_imp()

In [ ]:
feature.xgbR_imp()

### 4) Fine Tuning

In [ ]:
# Function for fine tuning an arbitrary model:

def tuning(X_train, y_train, modelo, params):
    
    cv = 5
    score = "neg_root_mean_squared_error"
    grid  = GridSearchCV(modelo, cv=cv, param_grid=params, 
                         scoring=score, 
                         n_jobs=-1,
                         return_train_score=True,
                         )

    grid.fit(X_train, y_train)

    best_index = grid.best_index_
    result = grid.cv_results_

    train_score = -result['mean_train_score'][best_index]
    left_out = -result['mean_test_score'][best_index]



    print(f"Train score: {train_score}")
    print(f"Left out data score: {left_out}")

    return grid.best_estimator_

In [ ]:
# Hypeparameters grid

params = {"n_estimators":[200, 300, 320],
          "criterion":["squared_error", "friedman_mse", "absolute_error", "poisson"],
          "max_depth":[2, 3, 4, 5, 6],
          "max_features":[0.5, 0.6, 0.8, "sqrt"],
          "min_samples_split":[3, 5, 6, 7]}

#### 4.1) No Feature Selection

In [ ]:
extra_tree = ExtraTreesRegressor(random_state=42)
best_estimator = tuning(X_train_rb, y_train, extra_tree, params)

#### 4.2) With Feature Selection

### 5) Predictions

#### 5.1) No feature Selection

In [ ]:
best_model = clone(best_estimator)
best_model.fit(X_train_rb, y_train)

In [ ]:
y_pred = best_model.predict(X_train_rb)

In [ ]:
rms_train = np.sqrt(mean_squared_error(y_pred, y_train))
r2_train = r2_score(y_pred, y_train)

print("Train set:")
print(f"RMSE: {rms_train}")
print(f"R2: {r2_train}")

In [ ]:
y_pred = best_model.predict(X_test_rb)
rmse_test = np.sqrt(mean_squared_error(y_pred, y_test))
r2_test = r2_score(y_pred, y_test)

print("Test set:")
print(f"RMSE: {rmse_test}")
print(f"R2: {r2_test}")

#### 5.2) With Feature Selection